In [1]:
from ete3 import Tree
import toytree
import toyplot
import toyplot.pdf
import itertools
import math
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import sys
pd.options.mode.chained_assignment = None

In [39]:
def nearest_node2leaf(phylo):
    node2leaf = {}
    for node in phylo.traverse(strategy="levelorder"):
        if not node.is_leaf():
            for child in node.get_children():
                if child.is_leaf():
                    node2leaf[child.name] = node.name
    return node2leaf

def branch_specific_length(sp, node2leaf, phylo):
    dist = phylo.get_distance(sp, node2leaf[sp])
    return dist

In [48]:
rate_phylo = Tree("./../trees/Mammals_phast.nh", format=1)
time_phylo = Tree("./../trees/mammals241.TimeTree.nwk", format=1)

Make sure all species in TimeTree are present in Zoonomia rate tree

In [49]:
time_species = time_phylo.get_leaf_names()

translate = {"Monachus_schauinslandi":None, 
             'Giraffa_camelopardalis':'Giraffa_tippelskirchi',
             'Murina_aurata':'Murina_feae',
             'Plecturocebus_donacophilus':None
            }
time_species = [translate[sp] if sp in translate else sp for sp in time_species]
time_species = [sp for sp in time_species if sp!=None]
rate_phylo.prune(time_species, preserve_branch_length=True)

In [50]:
rate_node2leaf = nearest_node2leaf(rate_phylo)
time_node2leaf = nearest_node2leaf(time_phylo)

In [65]:
species = ["Homo_sapiens","Mus_musculus", "Rattus_norvegicus","Canis_lupus","Felis_catus",'Mustela_putorius','Panthera_tigris','Bos_taurus','Equus_caballus','Colobus_angolensis']
for sp in species:
    time_dist = branch_specific_length(sp, time_node2leaf, time_phylo)
    rate_dist = branch_specific_length(sp, rate_node2leaf, rate_phylo)
    mut_per_year = (rate_dist/time_dist)/1e6
    print(mut_per_year, time_dist, rate_dist, sp)

8.970989662309115e-10 6.650905 0.00596652 Homo_sapiens
3.017378453429751e-09 3.06548222 0.00924972 Mus_musculus
4.1853762160179744e-09 20.8874174 0.0874217 Rattus_norvegicus
4.3101838052690826e-10 7.80599657 0.0033645280000000003 Canis_lupus
1.0187366010863221e-09 4.35333333 0.0044349 Felis_catus
1.5817428571428572e-09 17.5 0.0276805 Mustela_putorius
5.051642854034816e-10 7.41234111 0.00374445 Panthera_tigris
2.860629463956677e-09 0.62642157 0.00179196 Bos_taurus
4.4137065637065634e-10 2.59 0.00114315 Equus_caballus
7.3442265625e-10 12.8 0.00940061 Colobus_angolensis


KeyError: 'Giraffa_tippelskirchi'